# Joy Maps

An example of creating block diagrams inspired by Joy Division's 1979 *Unknown Pleasures* album cover using Kentucky's new state-wide Digital Surface Model created from recent Lidar data collection.

![Unknown Pleasures](https://www.typeroom.eu/assets/original/2020/05/18/JoyDivision-UnknownPleasures-B1.jpg)

Sources: KyFromAbove at https://kygeonet.ky.gov

In [ ]:
# Import ArcGIS Pro site package
import arcpy

In [ ]:
# Location of geodatabase
arcpy.env.workspace = 'c:/BoydsGIS/j/joy.gdb'

# Let's not add output to map
arcpy.env.addOutputsToMap = False

# Set our CRS
ky = arcpy.SpatialReference(3089)
wgs84 = arcpy.SpatialReference(4326)

In [ ]:
# Set our area of interest
lat = 38.1317719
lon = -84.5049721

# Distance in feet from lat,lon
buffer = 1000

In [ ]:
# KyFromAbove Rest endpoint from https://kygeonet.ky.gov/
dsm = 'https://kyraster.ky.gov/arcgis/rest/services/ElevationServices/Ky_DSM_First_Return_5FT_Phase1/ImageServer'

In [ ]:
# Create area of interest center point
point = arcpy.Point(lon, lat) 
ptGeometry = arcpy.PointGeometry(point, wgs84)
arcpy.env.outputCoordinateSystem = ky
arcpy.CopyFeatures_management(ptGeometry, "point")

In [ ]:
# Buffer and square our area of interest
arcpy.analysis.Buffer("point", "aoi", buffer)
arcpy.env.addOutputsToMap = True
arcpy.management.FeatureEnvelopeToPolygon('aoi', 'square_aoi')
arcpy.env.addOutputsToMap = False

In [ ]:
# Create collar for block diagram edges
arcpy.analysis.Buffer('square_aoi', 'clip', -10)
arcpy.analysis.Erase('square_aoi', 'clip', 'collar')

In [ ]:
# Extract DSM from area of interest
a = arcpy.sa.ExtractByMask(dsm, 'square_aoi')
a.save('dsm')
print(f"{a.minimum} is our lowest elevation. That's our floor")

In [ ]:
# Verify
b = arcpy.sa.Raster('dsm')
b

In [ ]:
# Create DSM surface with collared edges
arcpy.PolygonToRaster_conversion("collar", "ORIG_FID", "zero_raster", "#", "#", 5)
c = arcpy.sa.Con(arcpy.sa.IsNull('zero_raster'), b, b.minimum)
c.save('surface')

## Add line, then copy to fill area of interest

Create a new line feature class and add a single line that aligns with the north edge of the area of interest. Use snapping to make it exact.

![Create new feature class](https://nyc3.digitaloceanspaces.com/astoria/pro-relief/images/01.jpg)    
*Create new feature class*

![Set geometry to line](https://nyc3.digitaloceanspaces.com/astoria/pro-relief/images/02.jpg)    
*Set geometry to lines*

![Make sure if has correct CRS and finish](https://nyc3.digitaloceanspaces.com/astoria/pro-relief/images/03.jpg)    
*Make sure if has correct CRS and finish*


![Edit line and draw single feature on edge of AOI](https://nyc3.digitaloceanspaces.com/astoria/pro-relief/images/04.jpg)    
*Edit line and draw single feature on edge of AOI*


![Modify feature and copy parallel](https://nyc3.digitaloceanspaces.com/astoria/pro-relief/images/05.jpg)    
*Modify feature and copy parallel*


![Save and clear any selections](https://nyc3.digitaloceanspaces.com/astoria/pro-relief/images/06.jpg)    
*Save and clear any selections*

In [ ]:
# Add vertices along line every 5 feet
arcpy.edit.Densify('lines', '#', 5)

In [ ]:
# Add elevation to lines
arcpy.ddd.UpdateFeatureZ('lines', 'surface')

## Visualize

Add lines to New Local Scene and symbolize 3D line layer. Tip, add a fill symbology.

![Herrington Lake Dam](https://nyc3.digitaloceanspaces.com/astoria/pro-relief/images/end.jpg)    
*Herrington Lake Dam*